<a href="https://colab.research.google.com/github/gn1dus/mosi/blob/main/%D0%9B%D0%B0%D0%B1%D0%B0_3%D0%B3%D0%BF%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Нейронные сети для преобразования текста**

### Создать нейронную сеть с нуля, т.е. не используя готовые библиотеки. Пример работы на любом табличном датасете.


Функция для линейной регрессии

In [13]:
import numpy as np
import pandas as pd
def linear_regression(X: np.ndarray, weights: np.ndarray, bi) -> np.ndarray:
    return np.dot(X, weights) + bias

Функция для активации (сигмоид)

In [14]:
def activation_func(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

Функция для нейрона

In [4]:
def neuron(X: np.ndarray, weights: np.ndarray, bias: float) -> np.ndarray:
    temp_result = linear_regression(X, weights, bias)
    result = activation_func(temp_result)
    return result

Пример работы

In [15]:
data = {
        'feature1': [0.1, 0.2, 0.3, 0.4, 0.5],
        'feature2': [0.5, 0.4, 0.3, 0.2, 0.1],
        'label': [0, 0, 1, 1, 1]
    }

df1 = pd.DataFrame(data)

In [16]:
X = df1[['feature1', 'feature2']].values # Входные признаки
y = df1['label'].values # Целевая переменная

In [7]:
weights = np.random.rand(X.shape[1]) # Генерация весов
bias = np.random.rand() # Генерация смещений

In [17]:
output = neuron(X, weights, bias)
print(output)

[0.63375672 0.63796982 0.64216182 0.64633218 0.6504804 ]


### Сделать класс, в котором реализована возможность задать количество нейронов в скрытом слое и провести обучение.


In [21]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        # Инициализация весов и смещений
        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def activation_func(self, x):
        return 1 / (1 + np.exp(-x))  # Сигмоида

    def activation_func_derivative(self, x):
        return x * (1 - x)  # Производная сигмоиды

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activation_func(self.z1)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.activation_func(self.z2)

        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = self.activation_func(self.z3)

        return self.a3

    def backward(self, X, y):
        output_error = self.a3 - y  # Ошибка на выходе
        output_delta = output_error * self.activation_func_derivative(self.a3)

        hidden2_error = output_delta.dot(self.W3.T)
        hidden2_delta = hidden2_error * self.activation_func_derivative(self.a2)

        hidden1_error = hidden2_delta.dot(self.W2.T)
        hidden1_delta = hidden1_error * self.activation_func_derivative(self.a1)

        # Обновление весов и смещений
        self.W3 -= self.a2.T.dot(output_delta) * self.lr
        self.b3 -= np.sum(output_delta, axis=0, keepdims=True) * self.lr

        self.W2 -= self.a1.T.dot(hidden2_delta) * self.lr
        self.b2 -= np.sum(hidden2_delta, axis=0, keepdims=True) * self.lr

        self.W1 -= X.T.dot(hidden1_delta) * self.lr
        self.b1 -= np.sum(hidden1_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - self.a3))  # Среднеквадратичная ошибка
                print(f'Epoch {epoch}, Loss: {loss}')

Пример работы

In [22]:
# Подготовка данных
df2= pd.DataFrame(data)

X = df2[['feature1', 'feature2']].values
y = df2['label'].values.reshape(-1, 1)  # Преобразуем y в столбец

# Создаем и обучаем нейронную сеть
nn = NeuralNetwork(input_size=2, hidden1_size=3, hidden2_size=3, output_size=1, learning_rate=0.1)
nn.train(X, y, epochs=1000)

# Прогоняем данные через нейронную сеть
output = nn.forward(X)
print("Выход нейронной сети:", output)

Epoch 0, Loss: 0.25343579202470934
Epoch 100, Loss: 0.2400066653815962
Epoch 200, Loss: 0.24000605090893457
Epoch 300, Loss: 0.24000581026249956
Epoch 400, Loss: 0.24000557680809279
Epoch 500, Loss: 0.240005350185569
Epoch 600, Loss: 0.24000513006163615
Epoch 700, Loss: 0.24000491611605518
Epoch 800, Loss: 0.24000470804081556
Epoch 900, Loss: 0.24000450553935856
Выход нейронной сети: [[0.59996788]
 [0.59996429]
 [0.5999607 ]
 [0.59995711]
 [0.59995352]]
